# PartI-- uncompress files and combine all txt files into one txt file

In [104]:
#Change directory to where the ZIP file are stored
import os
import pandas as pd
import zipfile
import mysql.connector  
os.chdir('C:/Users/jessi/Desktop/comfortdelgro assignment')

In [105]:
path='C:/Users/jessi/Desktop/comfortdelgro assignment/2009.zip'
#with zipfile.ZipFile(path,"r") as zip_ref:
    #zip_ref..()
gz=[]
files_dir=zipfile.ZipFile(path)
files_list = files_dir.infolist()
for f in files_list:
    if f.filename.endswith('gz'):
        gz.append(f)
        

In [106]:
fact_data =pd.concat([pd.read_csv(files_dir.open(f),compression='gzip', header=0, delimiter="\t")
for f in files_list if f.filename.endswith('gz')])

In [107]:
fact_data=fact_data.rename(columns={'Date':"DateID"})

In [108]:
fact_data.shape

(1547897, 15)

In [109]:
fact_data.head()

,DateID,StoreID,StoreName,ProductID,ProductName,Color,Size,Manufacturer,OnHandQuantity,OnOrderQuantity,SafetyStockQuantity,UnitCost,DaysInStock,MinDayInStock,MaxDayInStock
0,2009-01-03,1,Contoso Seattle No.1 Store,6,Contoso 2G MP3 Player E200 Black,Black,1.6 x 0.4 x 3.7,"Contoso, Ltd",19,0,24,11.00,60,43,76
1,2009-01-03,1,Contoso Seattle No.1 Store,29,Contoso 32GB Video MP3 Player M3200 White,White,1 x 1.6 x 3.7,"Contoso, Ltd",19,0,12,84.49,58,11,97
2,2009-01-03,1,Contoso Seattle No.1 Store,31,Contoso 32GB Video MP3 Player M3200 Orange,Orange,2.6 x 2.5 x 0.6,"Contoso, Ltd",19,0,6,84.49,63,17,111
3,2009-01-03,1,Contoso Seattle No.1 Store,51,WWI 2GB Pulse Smart pen M100 Blue,Blue,6 x 0.5 x 0.5,Wide World Importers,19,0,6,91.95,79,12,86
4,2009-01-03,1,Contoso Seattle No.1 Store,54,WWI 4GB Video Recording Pen X200 Red,Red,5.1 x 2.4 x 3.9,Wide World Importers,23,4,18,98.07,30,30,71


In [110]:
final_data= fact_data[['DateID','StoreID','ProductID','OnHandQuantity','OnOrderQuantity',
             'DaysInStock', 'MinDayInStock','MaxDayInStock']]

In [111]:
final_data.shape

(1547897, 8)

In [112]:
final_data.to_csv('fact_table.csv',index=False)

# connect to  mysql database to compare primary keys&foreign keys for StoreID

In [169]:
#find errors with violating foreign key(StoreID) when uploading fact_table.csv into factstore in mysql database

In [113]:
  
cnx = mysql.connector.connect(user='root', password='Alicechen118a',
                              host='127.0.0.1',
                              database='dbo')

try:
    cursor = cnx.cursor()
    cursor.execute("""
      SELECT StoreID FROM dbo.dimstores;
   """)
    result = cursor.fetchall()
    #print(result)
finally:
    cnx.close()

In [114]:
Store_id = DataFrame(result,dtype=float)
Store_id.columns=cursor.column_names

In [115]:
Fact_storeID=pd.DataFrame(final_data['StoreID'].unique(),dtype=float)

In [116]:
Fact_storeID=Fact_storeID.rename(columns={0:"StoreID"})

In [117]:
[i for i in Fact_storeID.StoreID.tolist() if i not in Store_id.StoreID.tolist()]

[227.0]

In [118]:
final_data_v1=final_data[final_data.StoreID!=227]

In [119]:
final_data_v1.to_csv('fact_table_v2.csv',index=False)

In [121]:
final_data_v1.shape

(1543893, 8)

In [120]:
#in the end, upload this fact_table_v2 into mysql database dbo.factstore table